# 机器学习模型的评估

不管是怎样的机器学习的流程,你能控制的永远只有你能见到的东西.机器学习的目标是在前所未见的数据上取得准确的效果.在实现这个目标前,首先需要建立起对模型的评估.

大部分方法在前面的内容已经见过了.


## 训练集 验证集 测试集

前文的例子中,我们总是将可用数据拆分为 训练集 验证集 测试集

- 在验证集上进行模型超参数调整
- 在训练集上进行模型训练
- 最后在测试集上进行模型评估

为什么需要单独分出验证集?

- 进行模型训练前,最重要的一环是模型超参数的调整(模型的层数,层的大小,激活函数等等),选择超参数,需要一部分数据验证到底这样选,行不行.这部分数据就是验证集.
- 超参数调整虽然不是模型训练,当本质上还是一种学习,验证集的信息 "泄露" 到了超参数.如果在使用验证集训练模型,很快会过拟合.
  - 为了保证测试模型的效果,测试集的信息一点也不能泄露到模型本身.验证集数据绝对不能和测试集有交集.
  - 如果训练集包含验证集样本,这本身会加快过拟合,因此训练集也不能和验证集有交集.(这个不绝对)


### 简单留出验证

按照比例留出训练集 测试集.简单粗暴.

缺点

- 样本数量较少,没法进行确切划分
- 乱序对结果的影响很大.特别是在样本数量不足时.

[holdout_validation](./holdout_validation.png)


num_validation_samples = 10000 # 训练样本数量
np.random.shuffle(data) # 乱序
validation_data = data[:num_validation_samples]#测试集
training_data = data[num_validation_samples:]#训练集
model = get_model()
model.fit(training_data, ...)
validation_score = model.evaluate(validation_data, ...)

...

model = get_model()
model.fit(np.concatenate([training_data,
                          validation_data]), ...)
test_score = model.evaluate(test_data, ...)


### K 折验证

将数据分成 K 份,每次在第 i 个分区验证,其他分区训练.一般结果取平均值.

如果不同的训练-测试集划分评估结果差异很大,K 折验证能帮上忙.

但是与简单留出验证一样,也需要单独的验证集调整超参.

![k_fold_validation](./k_fold_validation.png)


In [ ]:
# k = 3 # 分 3 份
# num_validation_samples = len(data) // k #取划分后的数据集的长度
# np.random.shuffle(data) # 乱序
# validation_scores = []
# for fold in range(k):
#     validation_data = data[num_validation_samples * fold:
#                            num_validation_samples * (fold + 1)] # 测试集
#     training_data = np.concatenate(
#         data[:num_validation_samples * fold],
#         data[num_validation_samples * (fold + 1):]) # 训练集
#     model = get_model()
#     model.fit(training_data, ...)
#     validation_score = model.evaluate(validation_data, ...) # 评估
#     validation_scores.append(validation_score) # 存储每次的评估结果
# validation_score = np.average(validation_scores)# 平均评估
# model = get_model()
# model.fit(data, ...)# 训练
# test_score = model.evaluate(test_data, ...)

### 带有打乱数据的重复 K 折验证

如果你的数据很少,但又需要精确评估模型时,上乱序重复 K 折验证.

重复多次进行 K 折验证,每次新开始都要将样本重新乱序.

这样的代价很高,要训练 P * K 个模型.


## 常识性基线

虽然并不恰当,训练机器学习模型是一件概率性事件,也不是所有问题都适合机器学习解决.

如果机器学习的效果甚至不能打败一个极其简单的解决方案,那这样的模型毫无用途.

- mnist 数据集,随机化返回结果准确率 0.1
- imdb 数据集,随机结果是 0.5
- 二分类问题,如果样本不平衡,那随机分类器的准确度可能还要高.

当面对一个前人未解决问题时,一些非常简单的解决方案给定了一个基线.如果机器学习模型连基线都无法达到,此时需要的回退到问题之初重新思考.


## 评估模型的其他注意事项

数据的代表性

- 划分数据集时,随机性要够.否则 mnist 数据集数据按照类别排序,直接取很可能训练集压根没有 89.

时间箭头

- 所有与预测有关的问题,时间顺序不能打乱.测试集的任何数据都是要晚于训练集的,否则会造成信息的泄露.

数据冗余

- 数据集中某些样本可能会重复出现,如果恰好一个在训练集,一个在测试集.那完了,测试结果会变好,但是模型没用了.
